In [1]:
import pandas as pd
import os
import seaborn as sns
import json
import urllib.request

In [17]:
# retrieve RMSP TSE codes

ibge_tse = pd.read_csv('https://github.com/rafagarc/eleicoes_2022/raw/main/mapas_1o_turno/codigos_IBGE_TSE.csv')
rmsp = pd.read_html('https://pt.wikipedia.org/wiki/Regi%C3%A3o_Metropolitana_de_S%C3%A3o_Paulo')[3]
rmsp['Município'] = rmsp['Município'].apply(lambda x: x.upper().replace('[16]','') )
rmsp['Numero'] = rmsp.index.to_series()
rmsp = rmsp[['Numero', 'Município']]
rmsp = rmsp[rmsp['Município'] != 'TOTAL']
ibge_tse_sp = ibge_tse[ibge_tse.uf == 'SP']
ibge_tse_sp.columns = ['codigo_tse', 'uf', 'Município', 'capital', 'codigo_ibge']
rmsp_tse = rmsp.merge(ibge_tse_sp[['Município', 'codigo_tse']] , on='Município', how='left')
rmsp_tse.to_csv('regiao_metropolitana_de_sao_paulo_zonas_tse.csv', index='False')

In [76]:
# votes per zone capital

json_url = "https://resultados.tse.jus.br/oficial/ele2022/544/dados/sp/sp71072-c0001-e000544-v.json"

with urllib.request.urlopen(json_url) as url:
    data = json.load(url)
    
votes_per_zone = pd.concat([pd.DataFrame({'zona':[]} | {i['n']:[] for i in data['abr'][0]['cand']})])

for j in range(len(data['abr']))[1:]:
    zone = pd.DataFrame({'zona':[data['abr'][j]['cdabr']]} | {i['n']:[i['vap']] for i in data['abr'][j]['cand']})
    votes_per_zone = pd.concat([votes_per_zone, zone])

votes_per_zone_capital = votes_per_zone.reset_index(drop=True)

In [89]:
votes_per_zone_capital.head(10)

,zona,13,14,21,27,44,12,16,15,22,30,80
0,403,42308,55,66,24,701,3569,32,6033,30831,527,115
1,418,77028,82,98,26,1407,6431,54,10217,48437,1181,175
2,421,57642,64,61,22,871,4072,44,6178,39623,535,125
3,6,51419,73,124,24,414,5852,62,15715,48314,2377,118
4,405,42375,50,52,20,678,3784,28,5729,27404,433,130
5,371,93576,91,74,39,1406,5699,64,8141,41468,769,168
6,328,72614,87,79,38,1175,6105,72,8924,44340,998,157
7,348,26682,57,63,20,513,3649,30,6766,37638,815,75
8,353,76019,98,69,28,1440,4876,48,7084,45653,644,204
9,350,44006,62,69,27,740,4344,29,7290,43274,815,136


In [82]:
votes_per_zone = pd.concat([pd.DataFrame({'zona':[]} | {i['n']:[] for i in data['abr'][0]['cand']})])

for l in rmsp_tse.codigo_tse.to_list():

    json_url = f"https://resultados.tse.jus.br/oficial/ele2022/544/dados/sp/sp{l}-c0001-e000544-v.json"

    with urllib.request.urlopen(json_url) as url:
        data = json.load(url)

    for j in range(len(data['abr']))[1:]:
        zone = pd.DataFrame({'zona':[data['abr'][j]['cdabr']]} | {i['n']:[i['vap']] for i in data['abr'][j]['cand']})
        votes_per_zone = pd.concat([votes_per_zone, zone])

votes_per_zone_rmsp = votes_per_zone.reset_index(drop=True)

In [90]:
votes_per_zone_rmsp

,zona,21,22,14,15,12,13,44,30,80,16,27
0,335,36,26313,36,3186,1993,21271,348,361,56,8,12
1,386,48,44481,46,6526,4581,42470,572,976,95,23,15
2,199,59,48731,57,6719,5036,52164,706,732,115,13,33
3,319,6,9057,11,1169,506,6496,184,90,13,1,6
4,192,26,23497,45,3202,2182,24029,314,313,64,17,7
...,...,...,...,...,...,...,...,...,...,...,...,...
127,181,49,41779,66,5972,3494,40058,654,489,93,36,19
128,415,45,35618,61,5014,2966,33806,553,417,101,23,29
129,324,43,29165,38,5862,3798,34539,464,725,88,38,22
130,416,52,28449,56,5225,3486,55461,565,424,99,31,15


In [105]:
# Presidente RMSP: 2 turno

base_url = f"https://resultados.tse.jus.br/oficial/ele2022/545/dados/sp/sp71072-c0001-e000545-v.json"

with urllib.request.urlopen(base_url) as url:
    base_data = json.load(url)

votes_per_zone = pd.concat([pd.DataFrame({'zona':[]} | {i['n']:[] for i in base_data['abr'][0]['cand']})])

for l in rmsp_tse.codigo_tse.to_list():

    json_url = f"https://resultados.tse.jus.br/oficial/ele2022/545/dados/sp/sp{l}-c0001-e000545-v.json"

    with urllib.request.urlopen(json_url) as url:
        data = json.load(url)

    for j in range(len(data['abr']))[1:]:
        zone = pd.DataFrame({'zona':[data['abr'][j]['cdabr']]} | {i['n']:[i['vap']] for i in data['abr'][j]['cand']})
        votes_per_zone = pd.concat([votes_per_zone, zone])

votes_per_zone_rmsp_2t = votes_per_zone.fillna(0).astype('int32').reset_index(drop=True)

In [107]:
# Governador RMSP: 1 turno

base_url = f"https://resultados.tse.jus.br/oficial/ele2022/546/dados/sp/sp71072-c0003-e000546-v.json"

with urllib.request.urlopen(base_url) as url:
    base_data = json.load(url)

votes_per_zone = pd.concat([pd.DataFrame({'zona':[]} | {i['n']:[] for i in base_data['abr'][0]['cand']})])

for l in rmsp_tse.codigo_tse.to_list():

    json_url = f"https://resultados.tse.jus.br/oficial/ele2022/546/dados/sp/sp{l}-c0003-e000546-v.json"

    with urllib.request.urlopen(json_url) as url:
        data = json.load(url)

    for j in range(len(data['abr']))[1:]:
        zone = pd.DataFrame({'zona':[data['abr'][j]['cdabr']]} | {i['n']:[i['vap']] for i in data['abr'][j]['cand']})
        votes_per_zone = pd.concat([votes_per_zone, zone])

votes_per_zone_rmsp_1t_gov = votes_per_zone.fillna(0).astype('int32').reset_index(drop=True)

In [115]:
# Governador RMSP: 2 turno

base_url = f"https://resultados.tse.jus.br/oficial/ele2022/547/dados/sp/sp71072-c0003-e000547-v.json"

with urllib.request.urlopen(base_url) as burl:
    base_data = json.load(burl)

votes_per_zone = pd.concat([pd.DataFrame({'zona':[]} | {i['n']:[] for i in base_data['abr'][0]['cand']})])

for l in rmsp_tse.codigo_tse.to_list():

    json_url = f"https://resultados.tse.jus.br/oficial/ele2022/547/dados/sp/sp{l}-c0003-e000547-v.json"

    with urllib.request.urlopen(json_url) as url:
        data = json.load(url)

    for j in range(len(data['abr']))[1:]:
        zone = pd.DataFrame({'zona':[data['abr'][j]['cdabr']]} | {i['n']:[i['vap']] for i in data['abr'][j]['cand']})
        votes_per_zone = pd.concat([votes_per_zone, zone])

votes_per_zone_rmsp_2t_gov = votes_per_zone.fillna(0).astype('int32').reset_index(drop=True)

In [116]:
votes_per_zone_rmsp_2t_gov

,zona,10,13
0,405,0,0
1,348,0,0
2,421,0,0
3,418,0,0
4,328,0,0
5,6,0,0
6,403,0,0
7,371,0,0
8,249,0,0
9,353,0,0


dict